In [110]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time
import random
import itertools
from datetime import datetime
import torch.nn.functional as F

In [111]:
# set random seeds for reproducibility
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)

In [112]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# If we are on a CUDA machine, then this should print a CUDA device, but we are not, so it will run on CPU:
print(f'Working on device={device}')

Working on device=cpu


In [113]:
# Hyper-parameters

# each CIFAR image is RGB 32x32, so it is an 3D array [3,32,32]
# we will flatten the image as vector dim=3*32*32
input_size = 3*32*32

# we have 10 classes
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

num_classes = 10

num_epochs = 50
batch_size = 128

learning_rate = 0.01

In [114]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=batch_size,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [115]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 10)
        #self.fc3 = nn.Linear(84, 10)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


net = ConvNet()
net.to(device)

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

In [116]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [117]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    start_time = datetime.now()
    net.train()
    running_loss = 0.0
    epoch_loss = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # move data to device (GPU if enabled, else CPU do nothing)
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        epoch_loss += loss.item()

    epoch_loss = epoch_loss / len(trainloader)

    time_elapsed = datetime.now() - start_time

    # Test the model

    # set our model in the training mode
    net.eval()
    # In test phase, we don't need to compute gradients (for memory efficiency)
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Accuracy of the network on the 10000 test images
    acc = correct/total
    print(
        f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f} Test acc: {acc} time={time_elapsed}')


print('Finished Training')


Epoch [1/50], Loss: 1.7759 Test acc: 0.3931 time=0:00:28.687709
Epoch [2/50], Loss: 1.5782 Test acc: 0.4183 time=0:00:30.382453
Epoch [3/50], Loss: 1.5346 Test acc: 0.4596 time=0:00:29.551568
Epoch [4/50], Loss: 1.5090 Test acc: 0.4697 time=0:00:29.328534
Epoch [5/50], Loss: 1.4796 Test acc: 0.4432 time=0:00:29.057410
Epoch [6/50], Loss: 1.4781 Test acc: 0.4563 time=0:00:29.598949
Epoch [7/50], Loss: 1.4636 Test acc: 0.4714 time=0:00:29.765057
Epoch [8/50], Loss: 1.4496 Test acc: 0.4426 time=0:00:30.251707
Epoch [9/50], Loss: 1.4493 Test acc: 0.4601 time=0:00:29.903571
Epoch [10/50], Loss: 1.4262 Test acc: 0.4691 time=0:00:29.858458
Epoch [11/50], Loss: 1.4281 Test acc: 0.4839 time=0:00:30.007587
Epoch [12/50], Loss: 1.4262 Test acc: 0.4677 time=0:00:30.028156
Epoch [13/50], Loss: 1.4232 Test acc: 0.4872 time=0:00:29.919388
Epoch [14/50], Loss: 1.4071 Test acc: 0.4898 time=0:00:29.786341
Epoch [15/50], Loss: 1.4077 Test acc: 0.4894 time=0:00:30.037568
Epoch [16/50], Loss: 1.3872 Test a

In [ ]:
# Detailed accuracy per class
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))